In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d jangedoo/utkface-new

100% 331M/331M [00:02<00:00, 128MB/s]
100% 331M/331M [00:02<00:00, 128MB/s]


In [4]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [5]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [7]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
len(age)

23708

In [9]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [10]:
df.shape

(23708, 3)

In [11]:
df.head()

,age,gender,img
0,36,1,36_1_0_20170116165722892.jpg.chip.jpg
1,36,0,36_0_4_20170113133645566.jpg.chip.jpg
2,34,0,34_0_3_20170119200758796.jpg.chip.jpg
3,53,1,53_1_0_20170104184217989.jpg.chip.jpg
4,1,0,1_0_3_20161219225633088.jpg.chip.jpg


In [12]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [13]:
train_df.shape

(20000, 3)

In [14]:
test_df.shape

(3708, 3)

In [15]:
#data augumentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [17]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 1s 0us/step


In [19]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [20]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [21]:
model.compile(optimizer='adam',
              loss={'age': 'mae', 'gender': 'binary_crossentropy'},
              metrics={'age': 'mae', 'gender': 'accuracy'},
              loss_weights={'age':1,'gender':99})

In [22]:
model.fit(train_generator, batch_size=32, epochs=5, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 278s 427ms/step - loss: 101.6656 - age_loss: 15.3670 - gender_loss: 0.8717 - age_mae: 15.3670 - gender_accuracy: 0.5064 - val_loss: 85.2345 - val_age_loss: 16.6590 - val_gender_loss: 0.6927 - val_age_mae: 16.6590 - val_gender_accuracy: 0.5316
Epoch 2/10
159/625 [======>.......................] - ETA: 2:58 - loss: 84.7464 - age_loss: 15.3913 - gender_loss: 0.7006 - age_mae: 15.3913 - gender_accuracy: 0.5230

KeyboardInterrupt: 